In [9]:
library(sensitivity)

# Sobol' Indices Breakdown

## What are Sobol' Indices

Sobol' indices are a way to quantify the variance that random variables have on the output of a model. The 'best' way to use Sobol' indices is when you have a good understanding of the distributions of your input variables. Specifically, you want your inputs to come from 'nice' distributions. I am still figuring out these heuristics, but Heijungs 2024 seems to have some good ideas. He classifies Sobol' indices as 'uncertainty apportioning' (as opposed to uncertainty quantification or sensitivity analysis), which stuy the subdivision of output uncertainties in terms of the contributions by input uncertainties (see p. 695). Heijungs presents Morris's Elementary Effects as global UQ, but I am at this time unconvinced that it is in any way more useful than Sobol' indices. I need to examine it more though.

## First Order Effects

We can consider our output variable $Y$ to be some function of our input variables.
$$Y = f(X_1, X_2, ..., X_k)$$
Each input is a random variable. We can then decompose $f$ into terms of increasing dimension. Take $f_{ij} = f(X_i,X_j)$. Then,
$$f = f_0 = \sum_i f_i + \sum_{i<j}f_{ij} + ... + f_{12...k}$$
Sobol' proved that if each term in the above expansion has zero mean (e.g. $E[f_{247}] = 0$), then all the terms are orthogonal in pairs $E[f_if_j] = 0$. Note that if any function in your decomposition has non-zero mean $f_i = \mu$, you can build a new decomposition as follows:
$$f = (f_0 + \mu) + ... + (f_i - \mu) + ... = \tilde{f}_0 + ... + \tilde{f}_i + ...$$
with $E[\tilde{f}_i] = 0$. From the above, we can find the variances (written as $V$ here) of the decomposition terms. Starting with the first order terms:
$$E[Y | X_i] = f_0 + f_i = E[Y] + f_i$$
and so
$$V(f_i) = V(E[Y | X_i])$$
Divide by the unconditional variance to get the first order Sobol' indices.
$$S_i = \frac{V(E[Y | X_i])}{V(Y)}$$
Note that this term specifically gives us the variance from the $X_i$ function. Therefore, if $X_i$ is part of a higher order function (e.g. $f_{ij}$), the variance from that function will not be included in this Sobol' index.

## Higher Order Effects

 To get the higher order indices, we use a similar method. We must make sure we subtract off the variances we have already calculated.
$$V_{ij} = V\bigl(f_{ij}(X_i, X_j)\bigr) = V\bigl(\mathbb{E}[Y\mid X_i, X_j]\bigr)- V\bigl(\mathbb{E}[Y\mid X_i]\bigr)- V\bigl(\mathbb{E}[Y\mid X_j]\bigr)$$
Higher order terms follow in a similar fashion.

## Total Effects

Higher order indices often do not offer much in terms of explanatory power while demanding a large amount of computation. Therefore, total effects are employed to get the full of variance from each $X_i$ (i.e. the sum of its first order and higher order effects). The total effect of $X_1$ from $f(X_1,X_2,X_3)$ is given by
$$ S_{T1} = S_1 + S_{12} + S_{13} + S_{123} $$

## Computation

The standard computation of Sobol' indices uses a pseudo Monte-Carlo simulation. Build 2 matrices of $(N,k)$ dimension. Here, $N$ is (half) the number of samples you wish to generate, and $k$ is the length of our function $f$ (i.e. the number of input variables). Then, build $k$ matricies $C_i$ by replacing the $i$'th column of $B$ with the $i$'th column of $A$. Calculate the output for each row to build an output vector for each matrix. The first order indices are therefore calculated as follows.

$$ S_i \;=\; \frac{V\bigl[\mathbb{E}(Y\mid X_i)\bigr]}{V(Y)}
\;=\;
\frac{y_A\cdot y_{C_i} \;-\; f_0^2}{\,y_A\cdot y_A \;-\; f_0^2\,} $$

The total order indices are straightforwadly calculated as well.

$$ S_{T_i}
\;=\;
1 \;-\; \frac{V\bigl[\mathbb{E}(Y\mid X_{\sim i})\bigr]}{V(Y)}
\;=\;
1 \;-\; \frac{y_B \cdot y_{C_i} \;-\; f_0^2}{\,y_A \cdot y_A \;-\; f_0^2\,}$$

Here is explicity what the $A, B, C_i$ matrices look like. Note that $B$ uses a sample from $k+1$ to $2k$; however, this is just for notational simplicity. Matrix $B$ is from the exact same sample space as matrix $A$.

$$A = 
\begin{bmatrix}
x^{(1)}_{1} & x^{(1)}_{2} & \cdots & x^{(1)}_{i} & \cdots & x^{(1)}_{k} \\
x^{(2)}_{1} & x^{(2)}_{2} & \cdots & x^{(2)}_{i} & \cdots & x^{(2)}_{k} \\
\vdots      & \vdots      & \ddots & \vdots      &        & \vdots      \\
x^{(N)}_{1} & x^{(N)}_{2} & \cdots & x^{(N)}_{i} & \cdots & x^{(N)}_{k}
\end{bmatrix}, \quad
B = 
\begin{bmatrix}
x^{(1)}_{k+1} & x^{(1)}_{k+2} & \cdots & x^{(1)}_{k+i} & \cdots & x^{(1)}_{2k} \\
x^{(2)}_{k+1} & x^{(2)}_{k+2} & \cdots & x^{(2)}_{k+i} & \cdots & x^{(2)}_{2k} \\
\vdots        & \vdots        & \ddots & \vdots        &        & \vdots        \\
x^{(N)}_{k+1} & x^{(N)}_{k+2} & \cdots & x^{(N)}_{k+i} & \cdots & x^{(N)}_{2k}
\end{bmatrix},$$
$$ C_i = 
\begin{bmatrix}
x^{(1)}_{k+1} & x^{(1)}_{k+2} & \cdots & x^{(1)}_{i} & \cdots & x^{(1)}_{2k} \\
x^{(2)}_{k+1} & x^{(2)}_{k+2} & \cdots & x^{(2)}_{i} & \cdots & x^{(2)}_{2k} \\
\vdots        & \vdots        & \ddots & \vdots      &        & \vdots        \\
x^{(N)}_{k+1} & x^{(N)}_{k+2} & \cdots & x^{(N)}_{i} & \cdots & x^{(N)}_{2k}
\end{bmatrix} $$